In [1]:
import sys
sys.path.append("/home/ivan/Documents/GIT_PROJECTS/tfl-training-rl/src/")


In [5]:
!pip install pandas
%%capture

%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext training_rl

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


UsageError: Line magic function `%%capture` not found.


In [6]:
!pip install pandas
import warnings
import minari
from training_rl.offline_rl.behavior_policies.behavior_policy_registry import BehaviorPolicyType
from training_rl.offline_rl.custom_envs.custom_2d_grid_env.obstacles_2D_grid_register import ObstacleTypes
from training_rl.offline_rl.custom_envs.utils import Grid2DInitialConfig
from training_rl.offline_rl.generate_custom_minari_datasets.generate_minari_dataset_grid_envs import \
    create_minari_datasets, MinariDatasetConfig
from training_rl.offline_rl.custom_envs.custom_envs_registration import CustomEnv
from training_rl.offline_rl.custom_envs.custom_envs_registration import RenderMode
from training_rl.offline_rl.offline_policies.policy_registry import PolicyName
from training_rl.offline_rl.offline_trainings.offline_training import offline_training
from training_rl.offline_rl.offline_trainings.policy_config_data_class import TrainedPolicyConfig
from training_rl.offline_rl.custom_envs.utils import InitialConfigCustom2DGridEnvWrapper
from training_rl.offline_rl.custom_envs.custom_envs_registration import register_grid_envs
import gymnasium as gym
import torch
from training_rl.offline_rl.offline_trainings.policy_config_data_class import get_trained_policy_path
import os
from training_rl.offline_rl.offline_trainings.restore_policy_model import restore_trained_offline_policy
from training_rl.offline_rl.offline_policies.offpolicy_rendering import offpolicy_rendering
from training_rl.offline_rl.utils import delete_minari_data_if_exists
from minari import combine_datasets
from training_rl.offline_rl.load_env_variables import load_env_variables

load_env_variables()

warnings.filterwarnings("ignore")
# ToDo: this should be load automatically
register_grid_envs()

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


ModuleNotFoundError: No module named 'pandas'

restore our combined dataset of our previous notebook made of data collected from an optimal policy moving the agent from (4,0) to the target (7,7) and the other one given by a suboptimal stochastival trajectory bringing the agent from (0,0) to (7,0).

Our task is to create an optimal polica that be able to move the agent from (0,0) to (7,7). This was impossible for DQN as we saw before but as we saw in the theory BCQ and CQL are able to use the data in a much more efficient way so let's see if this is the case.

Let's restore the data and the environment from our previous exercise

In [ ]:
ENV_NAME = CustomEnv.Grid_2D_8x8_discrete
BEHAVIOR_POLICY_I = BehaviorPolicyType.behavior_8x8_moves_downwards_within_strip
NUM_STEPS_I = 3000
BEHAVIOR_POLICY_II = BehaviorPolicyType.behavior_8x8_eps_greedy_4_0_to_7_7
#BEHAVIOR_POLICY_II = BehaviorPolicyType.behavior_8x8_deterministic_4_0_to_7_7
NUM_STEPS_II = 1000

OFFLINE_POLICY = PolicyName.dqn

# Grid configuration
OBSTACLE = ObstacleTypes.verical_object_8x8
INITIAL_STATE = (0, 0)
FINAL_STATE = (7, 7)

env_2D_grid_initial_config = Grid2DInitialConfig(
    obstacles=OBSTACLE,
    initial_state=INITIAL_STATE,
    target_state=FINAL_STATE,
)

env = InitialConfigCustom2DGridEnvWrapper(gym.make(ENV_NAME, render_mode=RenderMode.RGB_ARRAY_LIST), env_config=env_2D_grid_initial_config)

In [ ]:
# Suboptimal policy
env_2D_grid_initial_config.initial_state=(4,0)
offpolicy_rendering(
    env_or_env_name=ENV_NAME,
    render_mode=RenderMode.RGB_ARRAY_LIST,
    behavior_policy_name=BEHAVIOR_POLICY_II,
    env_2d_grid_initial_config=env_2D_grid_initial_config,
    num_frames=1000,
)

In [ ]:
from training_rl.offline_rl.generate_custom_minari_datasets.generate_minari_dataset_grid_envs import \
    create_minari_config

DATA_SET_NAME = "data"
DATA_SET_IDENTIFIER_I = "_move_downwards"
DATA_SET_IDENTIFIER_II = "_move_deterministic"
VERSION_DATA_SET = "v0"

# Create metadata config for set I
minari_dataset_config_I = create_minari_config(
env_name=ENV_NAME,
dataset_name=DATA_SET_NAME,
data_set_identifier=DATA_SET_IDENTIFIER_I,
version_dataset=VERSION_DATA_SET,
num_steps=NUM_STEPS_I,
behavior_policy_name=BEHAVIOR_POLICY_I,
env_2d_grid_initial_config=env_2D_grid_initial_config
)

# Create metadata config for set II
minari_dataset_config_II = create_minari_config(
env_name=ENV_NAME,
dataset_name=DATA_SET_NAME,
data_set_identifier=DATA_SET_IDENTIFIER_II,
version_dataset=VERSION_DATA_SET,
num_steps=NUM_STEPS_II,
behavior_policy_name=BEHAVIOR_POLICY_II,
env_2d_grid_initial_config=env_2D_grid_initial_config
)

create_minari_datasets(minari_dataset_config_I)
create_minari_datasets(minari_dataset_config_II)

In [ ]:
from training_rl.offline_rl.generate_custom_minari_datasets.utils import generate_compatible_minari_dataset_name

MINARI_DATASET_I = "Grid_2D_8x8_discrete-data_verical_object_8x8_start_0_0_target_7_7_move_downwards-v0"
MINARI_DATASET_II = "Grid_2D_8x8_discrete-data_verical_object_8x8_start_0_0_target_7_7_move_deterministic-v0"
NAME_COMBINED_DATASET = "combined_data_sets_vertical_obstacle"

name_combined_dataset = generate_compatible_minari_dataset_name(
    env_name=ENV_NAME,
    data_set_name=NAME_COMBINED_DATASET,
    version="V0"
)

delete_minari_data_if_exists(name_combined_dataset)

list_dataset_names = [MINARI_DATASET_I, MINARI_DATASET_II]
minari_datasets = [
    minari.load_dataset(dataset_id) for dataset_id in list_dataset_names
]

combined_dataset = combine_datasets(
    minari_datasets, new_dataset_id=name_combined_dataset
)


print(f"Number of episodes in dataset I:{len(minari_datasets[0])}, in dataset I:{len(minari_datasets[1])} and  "
      f"in the combined dataset: {len(combined_dataset)}")


# Create metadata for the combined dataset (we can reuse the metadata of set I for simplicity)
minari_combined_dataset = MinariDatasetConfig.load_from_file(MINARI_DATASET_I)
minari_combined_dataset.num_steps = NUM_STEPS_I + NUM_STEPS_II
minari_combined_dataset.data_set_name = name_combined_dataset
minari_combined_dataset.save_to_file()

In [ ]:
NAME_EXPERT_DATA = name_combined_dataset

# The model policy to be trained.
POLICY_NAME = PolicyName.dqn

NUM_EPOCHS = 50
BATCH_SIZE = 256
UPDATE_PER_EPOCH = 100

# After every epoch we will collect some test statistics from the policy from NUMBER_TEST_ENVS independent envs.
NUMBER_TEST_ENVS = 1
EXPLORATION_NOISE = True
SEED = None #1626

# TrainedPolicyConfig is a handy object that will help us to deal with the policy configuration data.
offline_policy_config = TrainedPolicyConfig(
    name_expert_data=NAME_EXPERT_DATA,
    policy_name=POLICY_NAME,
    render_mode=RenderMode.RGB_ARRAY_LIST,
    device="cpu",
)


In [ ]:
# Run the training
offline_training(
    offline_policy_config=offline_policy_config,
    num_epochs = NUM_EPOCHS,
    number_test_envs=NUMBER_TEST_ENVS,
    update_per_epoch=UPDATE_PER_EPOCH,
    restore_training=False,
)

In [ ]:
POLICY_FILE = "policy_final.pth"
NUM_EPISODES = 10 # as more episodes the better

# restore a policy with the same configuration as the one we trained.
policy = restore_trained_offline_policy(offline_policy_config)
# load the weights
name_expert_data = offline_policy_config.name_expert_data
log_name = os.path.join(name_expert_data, POLICY_NAME)
log_path = get_trained_policy_path(log_name)
policy.load_state_dict(torch.load(os.path.join(log_path, POLICY_FILE), map_location="cpu"))


In [ ]:
offpolicy_rendering(
    env_or_env_name=env,
    render_mode=RenderMode.RGB_ARRAY_LIST,
    policy_model=policy,
    env_2d_grid_initial_config=env_2D_grid_initial_config,
    num_frames=1000,
    imitation_policy_sampling=False
)